<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;"> 
    <img src="https://upload.wikimedia.org/wikipedia/fa/thumb/f/fd/University_of_Tehran_logo.svg/1200px-University_of_Tehran_logo.svg.png" style="float:left;width:140px;padding-top: 10px" />
    <div style="font-size: 20px;padding-top: 30px;"><b>پروژه دو : الگوریتم ژنتیک</b></div>
    <div style="font-size: 16px;margin-top: 20px;">ثمین مهدی زاده 810196623</div>
</div>

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6"> 
در این پروژه سعی شده تا با استفاده از الگوریتم های ژنتیک که برگرفته از طبیعت و انتخاب طبیعی هستند پاسخ مناسب برای یک مسئله پیدا شود.
<br/>
شرح مسئله به این صورت است که نه گیت داریم که نحوه ی اتصال آن ها به این صورت است که گیت اول دو ورودی اول را می گیرد و بعد از آن هر گیت، خروجی گیت قبلی و ورودی اصلی بعدی مدار را می گیرد.همچنین جدول درستی ای داده شده است که خروجی های مطلوب را به ازای حالت های مختلف ده وروری نشان می دهد.هدف این پروژه پیدا کردن ترتیبی از گیت هاست به طوری که مطابق جدول درستی داده شده کار کند.
</div>

In [1]:
import pandas as pd
import random
import time

In [2]:
POPULATION_SIZE = 50
ELIMINATION_RATE = 20

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6"> 
در ابتدا جهت حل مدار کلاس زیر زده شده است.این کلاس تمام گیت های داده شده در صورت پروژه را پیاده سازی می کند و مشخص می کند خروجی هر گیت به ازای ورودی های آن چه خواهد بود.فیلد inputs ورودی های داده شده به مدار را می گیرد و فیلد gates نیز گیت های مدار را از بالا به پایین دریافت می کند.در انتها نیز dispathcher وجود دارد که هر گیت را به تابع مربوطه map می کند.
    <br/>
    برای محاسبه خروجی مدار نیز تابع calculate_output نوشته شده است که روی گیت ها حرکت می کند و خروجی نهایی را تولید می کند.
</div>

In [3]:
class Circuit:
    def __init__(self,gates,inputs):
        self.inputs = inputs
        self.gates = gates
        self.dispatcher = {"AND":self.AND,"OR":self.OR,"XOR":self.XOR,"NAND":self.NAND,"NOR":self.NOR,"XNOR":self.XNOR}
    
    def AND(self,a,b):
        if a and b:
            return True
        else:
            return False
        
    def OR(self,a,b):
        if a or b:
            return True
        else:
            return False
        
    def XOR(self,a,b):
        if a == b:
            return False
        else:
            return True
        
    def NAND(self,a,b):
        if a and b:
            return False
        else:
            return True
        
    def NOR(self,a,b):
        if a or b:
            return False
        else:
            return True
        
    def XNOR(self,a,b):
        if a == b:
            return True
        else:
            return False
        
    def calculate_output(self):
        out = self.dispatcher[self.gates[0]](self.inputs[0],self.inputs[1])
        for i in range(1,len(self.gates)):
            out = self.dispatcher[self.gates[i]] (out,self.inputs[i+1])
        return out

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6"> 
برای حل مسئله از کلاس solver استفاده شده است این کلاس با اجرای الگوریتم ژنتیک پاسخ مناسب برای مسئله را پیدا می کند.در ادامه چگونگی پیاده سازی این بخش آمده است. 
</div>

<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;"> 
    <b>مشخص کردن مفاهیم ژن و کروموزوم در مسئله</b>
</div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6;padding-bottom:3px;padding-top:8px;">
    گام اول برای حل یک مسئله جست و جو به روش ژنتیک، تعریف کروموزوم های مسئله می باشد. کروموزوم یک راه حل برای مسئله می باشد و از تعدادی ژن تشکیل شده است که بیانگر خصوصیات راه حل یا کروموزوم می باشند.
    <br/>
    هر ژن در این مسئله یکی از گیت های گفته شده در صورت پروژه است و کروموزم ها آرایه ای نه تایی از این گیت ها(ژن ها)هستند که نشان می دهد هر پاسخ دارای چه گیت هایی و به چه ترتیبی است.(عنصر اول آرایه اولین گیت و عنصر آخر آن،آخرین گیت مدار است)
    
</div>
<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;padding-bottom:3px;padding-top:5px;"> 
    <b>تولید جمعیت اولیه</b>
</div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    جمعیت اولیه برابر 50 در نظر گرفته شده است.چرا که زیاد بودن جمعیت سرعت محاسبه را کم می کند و کم بودن آن نیز تنوع موجود بین پاسخ ها را کاهش می دهد.برای تولید جمعیت اولیه تابع generate_chromosome زده شده است.این تابع به تعداد ژن های موجود در کروموزوم برای هر ژن به صورت تصادفی یک ژن از ژن های موجود را انتخاب می کند و یک کروموزم می سازد. 
    <br/>
    این تابع به تعداد جمعیت در نظر گرفته شده صدا زده می شود تا در نهایت تمام کروموزم های اولیه یک جمعیت ساخته شوند.
</div>
<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;padding-bottom:3px;padding-top:5px;"> 
    <b>پیاده سازی و مشخص کردن معیار سازگاری</b>
</div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    پس از به وجود آوردن جمعیت هر نسل باید به هر کدام از کروموزوم ها امتیازی اختصاص دهیم و بر اساس آن نسل بعدی را به وجود آوریم.
    <br/>
  برای اختصاص این معیار تابع   fitness پیاده سازی شده است این تابع به ازای هر کروموزوم محاسبه می کند که به چه تعداد از ورودی ها پاسخ درست داده نشده است یعنی کروموزومی که       fitnessکمتری دارد در اولویت بیشتری قرار می گیرد چرا که تعداد کمتری پاسخ غلط تولید کرده است.  
</div>
<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;padding-bottom:3px;padding-top:5px;"> 
    <b>پیاده سازی mutation و crossover و تولید جمعیت بعدی </b>
</div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6;padding-top:5px;">
    <li><b>Selection</b></li>
برای انتخاب جمعیت بعدی از rank base selection استفاده شده است.مزیت این روش به FPS این است که احتمال ها فاصله ی خیلی زیادی ندارند و این موضوع باعث می شود تا کروموزم هایی که احتمال کمتری دارند بتوانند در نسل بعد حضور یابند تا نسل بعدی خیلی یکنواخت نباشد.سپس بیست درصد جمعیت جدید را(ده کروموزوم)از بهترین کروموزوم های جمعیت قبل انتخاب می کنیم و مابقی را براساس احتمال هایی که به هر کروموزوم اختصاص داده بودیم دوبه دو انتخاب کرده و بچه های آن ها را وارد جمعیت جدید می کنیم.سپس بر روی هر کدام از بچه ها عملیات mutation را انجام می دهیم.
    <br/>
    لازم به ذکر است که در تلاش اول سعی شد که در ابتدا به تعداد جمعیت اولیه و با احتمال های داده شده جمعیتی انتخاب شود سپس از بین این جمعیت دو به دو برداشته شود و با احتمالی خود آن ها و با احتمال دیگر فرزندانشان وارد نسل جدید شوند اما زمان روش گفته شده در بالا به طور میانگین کمتر بود و به همین علت از آن استفاده شد.
    <li><b>Crossover</b></li>
    برای به وجود آوردن فرزندان از روش n-point-crossover استفاده شده است.به این منظور دو نقطه متمایز به صورت تصادفی روی هر کروموزوم انتخاب شده و به صورت یکی در میان بین هر کدام از این نقاط ژن های والد گذاشته شده است.تصادفی انتخاب کردن این نقاط تنوع فرزندان را بیشتر می کند و باعث می شود حتی اگر دوبار دو والد یکسان انتخاب شوند لزوما فرزندان یکسانی نداشته باشند.
     <li><b>Mutation</b></li>
mutation برای جلو گیری از یکنواخت شدن جمعیت انجام می شود.دراین الگوریتم هرژن با احتمالی می تواند جهش کند و به صورت تصادفی به جای آن ژن دیگری قرار بگیرد اما این احتمال متغیر است. در ابتدا احتمال در نظر گرفته شده برای جهش در هر جمعیت برابر fitness بهترین کروموزوم در جمعیت، تقسیم بر تعداد کل ورودی هاست(یعنی نسبت تعداد پاسخ های غلط بهترین کروموزم در جمعیت قبلی به کل ورودی ها)اما اگر پس از 30 بار اجرای الگوریتم بهترین پاسخ هر نسل تغییری نکرد احتمال را 0.3 زیاد کرده تا کروموزوم هایی با تنوع بیشتر در جمعیت بعدی حضور یابند اما ممکن است با این کار باز هم پاسخ نهایی تغییر نکند پس دوباره احتمال را کم کرده به امید آنکه به جواب برسد در واقع اگر پس از هر 30 بار پاسخ تغییری نکند احتمال جهش هر ژن را به صورت یکی در میان کم یا زیاد می کنیم.
</div>


In [4]:
class Solver:
    def __init__(self,data,gens_num):
        self.data = data
        self.gens = ['AND','OR','XOR','NAND','NOR','XNOR']
        self.gens_num = gens_num
        
    def generate_chromosome(self):
        chromosome = []
        for i in range(0,self.gens_num):
            chromosome.append(random.choice(self.gens))
        return chromosome
    
    def fitness(self,chromosome):
        count = 0
        for row in self.data:
            gens_out = Circuit(chromosome,row[:-1]).calculate_output()
            exp_out = row[-1]
            if(gens_out != exp_out):
                count+=1
        return count
    
    def sort_population_based_on_fitness(self,population):
        fitnesses = []
        for p in population:
            fitnesses.append(self.fitness(p))
        zipped_pairs = zip(fitnesses,population)
        sorted_population = [p for f,p in sorted(zipped_pairs)]
        return sorted_population
    
    def mutate(self,chromosome,p):
        mutate = []
        for i,gen in enumerate(chromosome):
            prob = random.random()
            if prob<p:
                mutate.append(random.choice(self.gens))
            else:
                mutate.append(chromosome[i])
        return mutate
                
    def cross_over(self,chromosome1,chromosome2):
        child1 = []
        child2 = []
        points = random.sample(range(1,self.gens_num-1),2)
        points.sort()
        for i in range(0,self.gens_num):
            if(points[0]>i):
                child1.append(chromosome1[i])
                child2.append(chromosome2[i])
            elif(points[1]<i):
                child1.append(chromosome1[i])
                child2.append(chromosome2[i])
            else:
                child1.append(chromosome2[i])
                child2.append(chromosome1[i])
        return child1,child2
        

    def genetic_algorithm(self):
        population = [self.generate_chromosome() for i in range(0,POPULATION_SIZE)]
        rank_sum = POPULATION_SIZE*(POPULATION_SIZE+1)/2
        probability = []
        for i in range(POPULATION_SIZE,0,-1):
            probability.append(i/rank_sum)
        count = 0
        prev_best = len(data)//2
        population = [self.generate_chromosome() for i in range(0,POPULATION_SIZE)]
        rank_sum = POPULATION_SIZE*(POPULATION_SIZE+1)/2
        probability = []
        for i in range(POPULATION_SIZE,0,-1):
            probability.append(i/rank_sum)
        count = 0
        prev_best = len(data)//2
        while True:
            population = self.sort_population_based_on_fitness(population)
            curr_best = self.fitness(population[0])
            
            if(curr_best == prev_best):
                count += 1
            else:
                count = 0
                
            p = curr_best/len(data)
            if(count>=30):
                if((count//20)%2 == 1 and p+0.3<=1):
                    p += 0.3

            if(self.fitness(population[0])==0):
                return population[0]
            
            new_population = []
            """
            selection = random.choices(population, weights=probability,k=POPULATION_SIZE)
            for i in range(POPULATION_SIZE//2):
                parents = random.sample(selection,2)
                prob = random.random()
                if(prob<=0.2):
                    new_population.append(parents[0])
                    new_population.append(parents[1])
                else:
                    child1,child2 = self.cross_over(parents[0],parents[1])
                    new_population.append(self.mutate(child1,p))
                    new_population.append(self.mutate(child2,p))
            """
            size = int((ELIMINATION_RATE*POPULATION_SIZE)/100) 
            new_population.extend(population[:size]) 
            size = int(((100-ELIMINATION_RATE)*POPULATION_SIZE)/100) 
            for i in range(size//2):
                parents = random.choices(population, weights=probability,k=2)
                child1,child2 = self.cross_over(parents[0],parents[1])
                new_population.append(self.mutate(child1,p))
                new_population.append(self.mutate(child2,p))
            population = new_population
            prev_best = curr_best 

In [5]:
data = pd.read_csv('truth_table.csv')
file = open('truth_table.csv', "r")
row_num = 0
data = []
for row in file:
    if(row_num!= 0):
        row = row.replace('\n','').split(',')
        row = [ True if x == "TRUE" else False for x in row]
        data.append(row)
    row_num+=1

In [6]:
start = time.time()   
solver = Solver(data,9)
print(solver.genetic_algorithm())
print("Time: %s seconds" % (time.time() - start))

['NAND', 'XNOR', 'OR', 'XNOR', 'AND', 'OR', 'NAND', 'XNOR', 'NOR']
Time: 16.420353174209595 seconds


<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;"> 
    <b>سوالات</b>
</div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    <li style="padding-bottom:5px;padding-top:8px;"><b> دلیل انتخاب معیار تناسب خود را ذکر کنید.</b></li>
    در این مسئله به دنبال پاسخی هستیم که به ازای همه ورودی ها پاسخ درست را برگرداند.بنابراین می توان این انتظار را داشت که کروموزمی که برای تعداد کمتری از ورودی ها پاسخ غلط می دهد کروموزوم بهتری بوده و مارا به جواب نزدیک تر می کند.
<li style="padding-bottom:5px;padding-top:8px;"><b>روش انتخاب افراد منتخب برای تولید جمعیت بعدی و دلیل انتخاب روش به کار برده شده را شرح دهید. </b></li>
    در قسمت مربوط به Selection روش های انتخاب شده به همراه دلیل انتخاب آن ها آورده شده است.
    <li style="padding-bottom:5px;padding-top:8px;"><b>تاثیر mutation و crossover و احتمال هر یک از آنها را ذکر کنید.</b></li>
    در mutation یک ژن به چیزی که از والد خود لزوما نگرفته است تغییر می کند. یعنی در هر بار عمل mutation یک جهش در یک یا چند ژن کروموزوم رخ می دهد که می تواند  یک جهش مثبت باشد و ما را به جواب نزدیک تر کند یا یک جهش منفی باشد و مارا از جواب دور کند. 
    <br/>
    در cross over فرزندان تولید شده همه ژن های خود را از والدین خود به ارث می برند و شانس یک ژن متفاوت با ژن های والد خود را ندارند و همواره مجبور به ارث بری از والدین خود دارند.
    <br/>
    در crossover تلاش می شود که با انتخاب والدین خوب ژن های آن ها به فرزندانشان منتقل شود بنابراین احتمالا این فرزندان نیز مارا به جواب نزدیک تر می کنند و احتمال یافتن ژن های خوب در آن ها بالا است.اما این عمل به تنهایی کافی نیست و گاهی لازم است برای نزدیک شدن به هدف جهش انجام شود چرا که ممکن است یکی از ژن های موجود در جواب در والدین نباشد و همین موضوع باعث نرسیدن به جواب شود. اما به طور کلی crossover از آن جا که انتقال دهنده ژن های خوب است با احتمال بیشتری نسبت به mutation انجام می شود.
    <br/>
    در این پروژه عملیات crossover روی%80 از جمعیت انجام می شود و mutation دارای احتمال متغیری است که راجع به آن در بخش های قبل(Mutation)صحبت شده است.
    <li style="padding-bottom:5px;padding-top:8px;"><b>با وجود استفاده از این روشها، باز هم ممکن است که کروموزومها پس از چند مرحله دیگر تغییر نکنند. دلیل این اتفاق و مشکلاتی که به وجود میآورد را شرح دهید. برای حل آن چه پیشنهادی میدهید؟ راه حلهای خود را امتحان کنید و بهترین آنها را روی پروژه خود پیاده سازی کنید. </b></li>
    ممکن است تفاوت بین احتمال های هر کروموزم به قدری زیاد باشد که تقریبا شانسی به کروموزوم های ضغیف تر داده نشود در صورتی که ممکن بود ترکیب ژن این کروموزوم با کروموزوم دیگر مارا به جواب نزدیک تر کند بنابراین یکی از این دلایل می تواند به دلیل انتساب احتمال نا مناسب به کروموزوم ها باشد ک باعث شود همواره دسته خاصی از کروموزوم ها انتخاب شود و روند تولید کروموزوم ها به سمت کروموزوم های مشابه برود و کروموزوم های با امتیاز پایین از رقابت کنار گذاشته شوند.همچنین انتخاب تابع crossover نیز اهمیت دارد اگر تابع crossover در تولید فرزندان تنوع نداشته باشد می تواند فرزندان تکراری را به جامعه اضافه کند.
    <br/>
    راه حل های درنظر گرفته شده که در این پروژه از آن استفاده شد به شرح زیر است:
    <li style="margin-right:20px">
    از rank base selection استفاده شده است تا تفاوت احتمالات را کاهش دهد(این اتفاق حین این که باعث می شود تنوع در جواب به وجود آید و به کروموزوم های ضعیف شانس انتخاب شدن را بدهد اما ممکن است از اهمیت کروموزوم های خوب کاسته شود به همین جهت ده درصد بهترین کروموزوم ها بدون هیچ احتمالی وارد نسل جدید می شوند تا همچنان از وجود کروموزوم های خوب مطمین باشیم)
    </li>
    <li style="margin-right:20px">
    برای crossover دو مرز انتخاب شده است که که این مرز ها ثابت نیستند و توسط تابع تصادفی به وجود می آیند در نتیجه تابع crossover با تنوع بیشتری می تواند فرزندان را به وجود آورد.
    </li>
    <li style="margin-right:20px">
    اگر روند تولید نسل متوجه شدیم تا چند نسل(در این جا 30)بهترین پاسخ دچار تغییر نشده احتمال جهش هر ژن را به صورت دوره ای کم با زیاد می کنیم.
    </li>
</div>